In [ ]:
import cv2
import os

# specify the directory where the mp4 files are located
path = 'Final_Data/MELD.Raw/output_repeated_splits_test'

# loop through each file in the directory
for filename in os.listdir(path):
    # check if the file is an mp4
    if filename.endswith('.mp4'):
        # open the mp4 file
        vidcap = cv2.VideoCapture(os.path.join(path, filename))
        success,image = vidcap.read()
        count = 0
        success = True
        
        folder_name = os.path.splitext(filename)[0]
        folder_path = os.path.join(path, folder_name)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        

        while success:
            success,image = vidcap.read()
            if success:
                # check if the image already exists
                img_path = os.path.join(path, folder_name, "img_%05d.jpg" % count)
                if not os.path.exists(img_path):
                    # save frame as a jpeg image
                    cv2.imwrite(img_path, image)
                count += 1

In [ ]:
import cv2
import os
import numpy as np
import random

def key_frame_extraction(frame_folder, output_dir, num_frames=8):
    frame_count = len(os.listdir(frame_folder))
    print(frame_count)
    if frame_count >= 80:
        # Use the key frame extraction code as it is
        fgbg = cv2.createBackgroundSubtractorMOG2()
        frame_count = 0
        key_frames = []
        
        for frame_name in sorted(os.listdir(frame_folder)):
            frame_path = os.path.join(frame_folder, frame_name)
            frame = cv2.imread(frame_path)
            
            if frame is not None:
                frame_count += 1
                fgmask = fgbg.apply(frame)
                key_frames.append((cv2.countNonZero(fgmask), frame, frame_name))
                    
        key_frames.sort(reverse=True, key=lambda x: x[0])
        key_frames = [(frame_name, frame) for count, frame, frame_name in key_frames[:num_frames]]
            
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        for i, (frame_name, key_frame) in enumerate(key_frames):
            img_name = f"img_{str(i).zfill(5)}"
            cv2.imwrite(os.path.join(output_dir, f'{frame_name}'), key_frame)
            
        print(f'Extracted {len(key_frames)} key frames from the video.')
    elif frame_count >= num_frames:
        # Select 10 random frames from the folder
        random_frames = random.sample(sorted(os.listdir(frame_folder), key=lambda x: x), num_frames)

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    
        for i, frame_name in enumerate(random_frames):
            frame_path = os.path.join(frame_folder, frame_name)
            frame = cv2.imread(frame_path)
            cv2.imwrite(os.path.join(output_dir, frame_name), frame)
        
        print(f'Selected {len(random_frames)} random frames from the video.')
    else:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        random_frames = [frame_name for frame_name in os.listdir(frame_folder) if frame_name.endswith(".jpg")]
        num_selected_frames = len(random_frames)

        for i, frame_name in enumerate(random_frames):
            frame_path = os.path.join(frame_folder, frame_name)
            frame = cv2.imread(frame_path)
            cv2.imwrite(os.path.join(output_dir, frame_name), frame)

        # If there are less than 8 images, we make additional random copies
        while len(os.listdir(output_dir)) < num_frames:
            random_index = random.randint(0, num_selected_frames - 1)
            frame_name = random_frames[random_index]
            frame_path = os.path.join(frame_folder, frame_name)
            frame = cv2.imread(frame_path)
            cv2.imwrite(os.path.join(output_dir, f"zopy_{len(os.listdir(output_dir))}_{frame_name}"), frame)

def rename_images(folder_path):
    for i, image_name in enumerate(sorted(os.listdir(folder_path))):
        old_path = os.path.join(folder_path, image_name)
        new_path = os.path.join(folder_path, f"img_{str(i).zfill(5)}.jpg")
        os.rename(old_path, new_path)
        
if __name__ == '__main__':
    train_splits_folder = 'dev_splits_complete'
    train_splits2_folder = 'final_dev_splits'

    for subfolder in os.listdir(train_splits_folder):
        frame_folder = os.path.join(train_splits_folder, subfolder)
        output_dir = os.path.join(train_splits2_folder, subfolder)
        
        key_frame_extraction(frame_folder, output_dir)
        rename_images(output_dir)

In [ ]:
import os
import cv2

def convert_to_grayscale(root_folder):
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                filepath = os.path.join(dirpath, filename)
                image = cv2.imread(filepath)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                resized = cv2.resize(gray, (256, 256))
                cv2.imwrite(filepath, resized)
                
convert_to_grayscale("final_test_splits")